# Law, Bias, and Algorithms
## Narrow Tailoring and Disparate Impact in Law School Admissions
In this exercise, we'll examine admissions decisions at top-tier law schools using the dataset from the _LSAC National Longitudinal Bar Passage Study_ ([Wightman and Ramsey, 1998](https://files.eric.ed.gov/fulltext/ED469370.pdf)).
This study presents national longitudinal bar passage data gathered from the class that started law school
in fall 1991 over a 5-year period.
In our analysis, we will focus on diversity and affirmative action policies. We'll explore a simple method to reverse engineer admissions criteria, and investigate the extent to which race-blind policies can achieve diversity. We'll also consider the consequences on diversity of a hypothetical scenario in which admissions decisions are based on statistical likelihood of bar passage.

In [0]:
# Some initial setup
options(digits = 3)
library(tidyverse)
theme_set(theme_bw())

# Read the data
bar_data <- read_csv("../data/bar_passage_data.csv", 
                 col_types = cols(MINORITY="l", TOP_TIER="l", MALE="l", PASS_BAR="l")) %>% 
    mutate(FAM_INC = as.factor(FAM_INC))

Each row in the data corresponds to a law school admit. The dataset contains the following variables:

* An ID number:
    * `ID`
    
    
* Base demographic information about the applicant:
    * `MINORITY` is encoded as follows:        
        * `False`: Non-hispanic White
        * `True`: Asian, Black, Hispanic, American Indian, Alaskan Native, or Other
    * `MALE` is coded as `True` for male applicants and `False` for female applicants
        
        
* Outcome of interest, Bar Passage:
    * `PASS_BAR` is an indicator variable and is encoded as 0 regardless of why the student did not pass the exam.  They may have dropped out of law school, never taken the bar, or failed the exam. `PASS_BAR` is encoded as 1 if the student eventually passes the bar. 
    * `BAR` provides more detail about bar results and test history
    
 
* Academic Indicators:
    * `UGPA` (undergraduate GPA), `LSAT` (LSAT score, scaled to be between 10 and 50)
    
    
* Tier of Law School Attended:
    * `TOP_TIER` is an indicator variable for whether an applicant ultmiately attends a top tier school
    * Note that students who attend historically black colleges and universities were removed as those schools are outliers in law school admissions.


* Family Income Quintile:
    * `FAM_INC` provides the family income quintile
    * `FAM_INC_1`, `FAM_INC_2`, `FAM_INC_3`, `FAM_INC_4`,` FAM_INC_5` are indicator variables for the income quintile

Law school admits whose entries had missing data have been removed.

### Exploratory Data Analysis

We start our analysis by exploring class composition and racial disparities.

#### Exercise 1: Demographic Composition and Disparities

1. Create a table showing the total number of law school admits, the number of minority admits, and the percentage of law school admits who are minorties.
2. Recalculate the statistics above, but for top-tier law schools only.
3. Compute the average LSAT and undergraduate GPA by minority status.

In [0]:
# WRITE CODE HERE


We note that the majority-minority test gap has been the subject of extensive scientific inquiry. Potential causes include differences in school resources, poverty, family structure, environment, and discrimination.

### Reverse Engineering Current Admissions

We now attempt to reverse engineer admissions criteria for top-tier law schools. To do so, we make three key assumptions. First, we assume that students in our dataset comprise the full set of students who _applied_ to law school. In reality, our dataset only contains students who ultimately enrolled at a law school. Second, we assume that [students accepted to top-tier law schools](https://abovethelaw.com/2013/03/which-law-schools-had-the-highest-yield-rate/) all decided to enroll at a top-tier school. Finally, we assume that admissions decisions are based on a relatively small set of factors that we have access to: LSAT score, GPA, minority status, and family income. This is a coarse approximation of actual admissions policies, but is instructive nevertheless.

Given these assumptions, we can try to reconstruct admissions policies by fitting a simple logistic regression model that predicts acceptance to a top-tier school based on the available information. 

In R, you can specify statistical models using formulas of the form `outcome variable ~ input variables` with each input variable seperated with the `+` symbol. We'll learn more about these models in the coming weeks, but for now we'll treat them (mostly) as black boxes.

In [0]:
# fit a logistic regression to predict acceptance at a top-tier school
lr_admit <- glm(TOP_TIER ~ LSAT + UGPA + MINORITY + FAM_INC_1 + FAM_INC_2, 
                    data = bar_data, family="binomial")

# summarize the model
summary(lr_admit)

The list above shows the coefficient for each covariate estimated by our logistic regression model. We can think of the coefficients as indicating how much different factors are weighted when making admissions decisions.

#### Exercise 2: 
Discuss the meaning of this model. What does it say about how law schools are admitting students? How accurate do you think it is? In what ways do you think it is misrepresenting or simplyifing the law school admissions process?

### Simulating Law School Admissions

#### Exercise 3: Exploring Alternative Admissions Policies

You'll now create an algorithm for admitting students to top-tier schools based on any given weighting of LSAT, GPA, minority status, and low-income status. Once the weights are provided, the algorithm should sort all the applicants and return the subset of $n$ = 6,882 applicants ranked highest, where $n$ is the actual number admitted to the top-tier schools.

Explore various admissions policies. Are you able to create admissions criteria that match the quality and diversity of the applicant pool? Are you able to do so without explictly using race? Recall that _Gratz_ declared using race in a points based way as part of college admissions unconstitutional. 

In [0]:
# WRITE CODE HERE
admit_n <- sum(bar_data$TOP_TIER)

# weights inferred from the logistic regression above.
# these can be modified to explore alternative policies
LSAT_wt <- 0.16
GPA_wt <- 1
MINORITY_wt <- 1.3
INC1_wt <- 0.38
INC2_wt <- 0



### Using Predicted Bar Passage as a Selection Criterion

Finally, we consider what would happen if law schools selected students to optimize bar passage rates. This approach might be motivated from two perspectives. First, perhaps using an outcome-based algorithm would allow schools to lessen the weight on LSAT scores, given the critiques of standardized tests as favoring affluent non-minority groups, and hence constitute a "workable race-neutral alternative." Second, more crudely, one of the major inputs into U.S News and World Report law school rankings is bar passage. Schools might want to admit a class to increase bar passage rates or U.S. News might increase the weight of bar passage in its rankings. Our goal here is to examine whether the adoption of such a policy is a workable alternative and whether it might have disparate impact.

#### Exercise 4:

Create a model to predict bar passage and then use this model to simulate an admissions cycle where the students predicted as being the most likely to pass the bar are admitted into the highest tier law schools. Create the predictive model using logistic regression as shown above.

Suppose an admissions office came to you and proposed using this model to determine which students are admitted. How would you evaluate the model and what would you recomemnd to the admissions office? If this model were used, would there be a valid disparate action claim for any rejected applicants?

In [0]:
# WRITE CODE HERE


#### Discussion Questions

* One way to characterize the use of bar passage information is as an attempt to reduce the importance of the LSAT in determining law school admissions. Does using bar passage data fulfill the goal of reducing emphasis on the LSAT?

* Consider what some of the potential problems with this dataset are. What factors are not represented in the data that might be relevant for predicting outcomes on the bar exam? For success as an attorney? Are their any concerns about state bar passage as an outcome measure?

* How well do these models mimic the procedure of the actual admissions process? How does the performance of actual admission officers compare to the models we have here and to the extent there are differences in outcomes, what factors might drive those differences? 

* Are there important differences between the populations of interest that may influence the model in undesirable ways? Consider whether minority students are more likely to practice in jurisdictions with lower bar passage rates (e.g., NY or CA)? Consider whether stereotype threat or implicit bias might explain differences in academic or bar passage performance between white and minority students and what implications that has for the approach you've studied above.